### Minimum Spanning Tree

&nbsp;

Minimum spanning tree is a subset of a graph, where every vertex is connected to at least one other vertex, but at most connected to two other vertices, that indicates no cycle, and the total weight of the graph is the minimum possible. Lol, long definition!

&nbsp;

In [1]:
import os
os.chdir('K:/ecole/github')

#graph adt
#check the below link for more details
# https://github.com/je-suis-tm/graph-theory/blob/master/graph.py
import graph

In [2]:
#we need an undirected graph
#in another word, vertices with edge connections 
#are mutually connected to each other
ADT=graph.graph()

ADT.append(1,2,6)
ADT.append(1,3,5)
ADT.append(2,1,6)
ADT.append(2,4,8)
ADT.append(2,6,3)
ADT.append(3,1,5)
ADT.append(3,4,2)
ADT.append(3,5,7)
ADT.append(4,2,8)
ADT.append(4,3,2)
ADT.append(4,5,7)
ADT.append(5,3,3)
ADT.append(5,4,7)
ADT.append(5,7,9)
ADT.append(6,2,3)
ADT.append(6,7,5)
ADT.append(7,5,9)
ADT.append(7,6,5)
ADT.append(7,8,13)
ADT.append(8,7,13)

![alt text](./preview/minimum%20spanning%20tree%20origin.jpg)

### Prim's Algorithm

&nbsp;

Prim's algorithm is perfect for a minimum spanning tree problem. The algorithm is somewhat similar to BFS. We start from one vertex and check its child vertices. In BFS, we pop vertices by left to right order. In Prim, we only pop the vertex with minimum weight and ignore the rest. When all available vertices have been popped, we obtain a minimum spanning tree.

Details of Prim's algorithm can be found in the following link

http://interactivepython.org/runestone/static/pythonds/Graphs/PrimsSpanningTreeAlgorithm.html


Details of BFS can be found in the following link

https://github.com/je-suis-tm/graph-theory/blob/master/BFS%20DFS%20on%20DCG.ipynb

&nbsp;

In [3]:
#different starting point may end up with a different tree
def prim(ADT,start):
    """Prim's Algorithm to find a minimum spanning tree"""
    
    #we use a dictionary instead of a list as queue
    #cuz we need to pop the vertex with minimum weight on the edge
    queue={}
    queue[start]=0
    
    #route keeps track of how we travel from one vertex to another
    route={}
    route[start]=start
    
    #result is a list that keeps the order of vertices we have visited
    result=[]
    
    while queue:
                
        #note that when we have two vertices with the same minimum weights
        #the dictionary would pop the one with the smallest key
        current=min(queue,key=queue.get)
        queue.pop(current)
        result.append(current)
        ADT.visit(current)
        
        #BFS
        for i in ADT.edge(current):
            if i not in queue and ADT.go(i)==0:
                queue[i]=ADT.weight(current,i)
                route[i]=current
                
            #every time we find a smaller weight
            #we need to update the smaller weight in queue
            if i in queue and queue[i]>ADT.weight(current,i):
                queue[i]=ADT.weight(current,i)
                route[i]=current               
    
    #create minimum spanning tree
    subset=graph.graph()
    for i in result:
        if i!=start:
            subset.append(route[i],i,ADT.weight(route[i],i))
            subset.append(i,route[i],ADT.weight(route[i],i))

    return subset

In [4]:
MST=prim(ADT,1)

ADT.clear(whole=True)

MST.reveal()

{1: {3: 5, 2: 6},
 3: {1: 5, 4: 2, 5: 7},
 4: {3: 2},
 2: {1: 6, 6: 3},
 6: {2: 3, 7: 5},
 7: {6: 5, 8: 13},
 5: {3: 7},
 8: {7: 13}}

![alt text](./preview/minimum%20spanning%20tree%20subset.jpg)

### Kruskal's Algorithm

&nbsp;

Kruskal is another algorithm to get minimum spanning tree. Unlike Prim, Kruskal cannot select where to expand the tree. Kruskal requires all edges in the graph ADT to be sorted. Those edges are stored inside a queue by ascending order. The algorithm iterates through the queue and picks out the edge with the smallest weight, as long as the edge doesn't create a cycle in the subset, the edge will be added into the subset. When the iteration stops, a minimum spanning tree is born.

&nbsp;

In [5]:
#use disjoint set to detect cycle
def trace_root(disjointset,target):
    """Use recursion to trace root in a disjoint set"""

    if disjointset[target]!=target:
        trace_root(disjointset,disjointset[target])
    else:
        return target

In [6]:
#unlike prim, kruskal does not require starting vertex
#it always starts with the edge with the minimum weight
def kruskal(ADT):
    """Kruskal's Algorithm to find the minimum spanning tree"""
    
    #use dictionary to sort edges by weight
    D={}  
    for i in ADT.vertex():
        for j in ADT.edge(i):
            
            #convert edge into string
            #as the graph is bidirected
            #we only need one edge for each pair of two vertices
            if f'{j}-{i}' not in D.keys():
                D[f'{i}-{j}']=ADT.weight(i,j)
                
    sort_edge_by_weight=sorted(D.items(), key=lambda x:x[1])    
    
    result=[]
    
    #to achieve minimum spanning tree
    #we need to avoid cycles
    #here we use disjointset to detect cycle
    #for more details, you can go to geeksforgeeks
    # https://www.geeksforgeeks.org/union-find/
    disjointset={}
    
    #lets skip the part where default=-1
    for i in ADT.vertex():
        disjointset[i]=i
        
    for i in sort_edge_by_weight:
        
        parent=int(i[0].split('-')[0])
        child=int(i[0].split('-')[1])
                
        #first we need to check disjoint set
        #if it already has indicated cycle
        #trace_root function will go to infinite loops
        if disjointset[parent]!=disjointset[child]:
            
            #if we trace back to the root of the tree
            #and it indicates no cycle
            #we update the disjoint set and add edge into result
            if trace_root(disjointset,parent)!=trace_root(disjointset,child):
                disjointset[child]=parent
                result.append([parent,child])                
                
    #create minimum spanning tree
    subset=graph.graph()
    for i in result:
        subset.append(i[0],i[1],ADT.weight(i[0],i[1]))
        subset.append(i[1],i[0],ADT.weight(i[0],i[1]))

    return subset

In [7]:
MST=kruskal(ADT)

ADT.clear(whole=True)

MST.reveal()

{3: {4: 2, 1: 5, 5: 7},
 4: {3: 2},
 2: {6: 3, 1: 6},
 6: {2: 3, 7: 5},
 1: {3: 5, 2: 6},
 7: {6: 5, 8: 13},
 5: {3: 7},
 8: {7: 13}}

![alt text](./preview/minimum%20spanning%20tree%20subset.jpg)

### Borůvka's Algorithm
&nbsp;

Borůvka is the oldest algorithm for minimum spanning tree. The logic is very similar to Kruskal. We can almost say Kruskal is based upon Borůvka. Basically for each vertex, we try to find its edge with the minimum weight
after the first iteration. We will end up with a few connected components. The second iteration is similar to the first. For each pair of connected components, we try to find the edge with minimum weight to connect. After two rounds of iterations, we are left with a minimum spanning tree.

&nbsp;

In [8]:
#borůvka requires another algorithm to detect connected components
#here i use dfs
def boruvka(ADT):    
    """Borůvka's Algorithm to find the minimum spanning tree"""
    
    subset=graph.graph()
    
    #get the edge with minimum weight for each vertex
    for i in ADT.vertex():
        minimum=float('inf')
        target=None
        for j in ADT.edge(i):
            if ADT.weight(i,j)<minimum:
                minimum=ADT.weight(i,j)
                target=[i,j]
        
        #as the graph is undirected
        #we append both edges
        subset.append(target[0],target[1],ADT.weight(target[0],target[1]))
        subset.append(target[1],target[0],ADT.weight(target[0],target[1]))

    
    #use dfs topological sort to find connected components
    #details of topological sort can be found in the following link
    # https://github.com/je-suis-tm/graph-theory/blob/master/topological%20sort.ipynb
    connected_components=[]
    for i in subset.vertex():
        
        #avoid duplicates of connected components
        #use jump to break out of multiple loops
        jump=False
        for j in connected_components:
            if i in j:
                jump=True
                break
        if jump:
            continue
        connected_components.append(list(graph.dfs_topo_sort(subset,i)))
    
    #connect 2 connected components with minimum weight
    #same logic as the first iteration
    for i in range(len(connected_components)):
        for j in range(i+1,len(connected_components)):
            minimum=float('inf')
            target=None
            for k in connected_components[i]:
                for l in ADT.edge(k):
                    if l in connected_components[j]:
                        if ADT.weight(k,l)<minimum:
                            minimum=ADT.weight(k,l)
                            target=[k,l]
            
            subset.append(target[0],target[1],minimum)
            subset.append(target[1],target[0],minimum)
                  
    return subset

In [9]:
MST=boruvka(ADT)

ADT.clear(whole=True)

MST.reveal()

{1: {3: 5, 2: 6},
 3: {1: 5, 4: 2, 5: 3},
 2: {6: 3, 1: 6},
 6: {2: 3, 7: 5},
 4: {3: 2},
 5: {3: 3},
 7: {6: 5, 8: 13},
 8: {7: 13}}

![alt text](./preview/minimum%20spanning%20tree%20subset.jpg)

### Time Complexity Comparison
&nbsp;

See how Prim revolutionized the minimum spanning tree?

&nbsp;

In [10]:
%%timeit

MST=graph.prim(ADT,1)

ADT.clear(whole=True)

45.6 µs ± 523 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [11]:
%%timeit

MST=graph.kruskal(ADT)

ADT.clear(whole=True)

61 µs ± 723 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [12]:
%%timeit

MST=graph.boruvka(ADT)

ADT.clear(whole=True)

64.1 µs ± 1.01 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
